In [ ]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 660.5/660.5 kB 9.0 MB/s eta 0:00:00


In [ ]:
import os
import shutil
import random

!pip install tqdm --upgrade
from tqdm.notebook import tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_path_img = "./yolo_data/images/train/"
train_path_label = "./yolo_data/labels/train/"
val_path_img = "./yolo_data/images/val/"
val_path_label = "./yolo_data/labels/val/"
test_path = "./yolo_data/test"

In [ ]:
'''
Split the dataset into train and test and creates the train.txt and test.tx with
the respective path of the images in each folder
'''

def train_test_split(path,neg_path=None, split = 0.2):
    print("------ PROCESS STARTED -------")


    files = list(set([name[:-4] for name in os.listdir(path)])) ## removing duplicate names i.e. counting only number of images


    print (f"--- This folder has a total number of {len(files)} images---")
    random.seed(42)
    random.shuffle(files)

    test_size = int(len(files) * split)
    train_size = len(files) - test_size

    ## creating required directories

    os.makedirs(train_path_img, exist_ok = True)
    os.makedirs(train_path_label, exist_ok = True)
    os.makedirs(val_path_img, exist_ok = True)
    os.makedirs(val_path_label, exist_ok = True)


    ### ----------- copying images to train folder
    for filex in tqdm(files[:train_size]):
      if filex == 'classes':
          continue

      # Check if either ".jpg" or ".jpeg" file exists and copy accordingly
      if os.path.isfile(os.path.join(path, filex + '.jpg')):
          shutil.copy2(os.path.join(path, filex + '.jpg'), f"{train_path_img}/" + filex + '.jpg')
      elif os.path.isfile(os.path.join(path, filex + '.jpeg')):
          shutil.copy2(os.path.join(path, filex + '.jpeg'), f"{train_path_img}/" + filex + '.jpeg')

      # Assuming corresponding label files have the same extension as the images
      # If not, modify this part accordingly
      if os.path.isfile(os.path.join(path, filex + '.txt')):
          shutil.copy2(os.path.join(path, filex + '.txt'), f"{train_path_label}/" + filex + '.txt')




    print(f"------ Training data created with 80% split {len(files[:train_size])} images -------")

    if neg_path:
        neg_images = list(set([name[:-4] for name in os.listdir(neg_path)])) ## removing duplicate names i.e. counting only number of images
        for filex in tqdm(neg_images):
            shutil.copy2(neg_path+filex+ ".jpg", f"{train_path_img}/" + filex + '.jpg')

        print(f"------ Total  {len(neg_images)} negative images added to the training data -------")

        print(f"------ TOTAL Training data created with {len(files[:train_size]) + len(neg_images)} images -------")



    ### copytin images to validation folder
    for filex in tqdm(files[train_size:]):
      if filex == 'classes':
          continue

      # Check if either ".jpg" or ".jpeg" file exists and copy accordingly
      if os.path.isfile(os.path.join(path, filex + '.jpg')):
          shutil.copy2(os.path.join(path, filex + '.jpg'), f"{val_path_img}/" + filex + '.jpg')
      elif os.path.isfile(os.path.join(path, filex + '.jpeg')):
          shutil.copy2(os.path.join(path, filex + '.jpeg'), f"{val_path_img}/" + filex + '.jpeg')

      # Assuming corresponding label files have the same extension as the images
      # If not, modify this part accordingly
      if os.path.isfile(os.path.join(path, filex + '.txt')):
          shutil.copy2(os.path.join(path, filex + '.txt'), f"{val_path_label}/" + filex + '.txt')


    print(f"------ Testing data created with a total of {len(files[train_size:])} images ----------")

    print("------ TASK COMPLETED -------")

## spliting the data into train-test and creating train.txt and test.txt files
# train_test_split('/content/drive/MyDrive/custom_notebooks/yolo_data/')

path = '/content/drive/MyDrive/yolov8/data/'
train_test_split(path)

------ PROCESS STARTED -------
--- This folder has a total number of 89 images---


  0%|          | 0/72 [00:00<?, ?it/s]

------ Training data created with 80% split 72 images -------


  0%|          | 0/17 [00:00<?, ?it/s]

------ Testing data created with a total of 17 images ----------
------ TASK COMPLETED -------


# New section

In [20]:

import ultralytics
ultralytics.checks()


Ultralytics YOLOv8.0.227 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 27.0/78.2 GB disk)


In [22]:
!yolo task=detect mode=train model=yolov8s.pt data=/content/drive/MyDrive/yolov8/dataset.yaml epochs=10 imgsz=640 batch=8 project=/content/drive/MyDrive/yolov8/training_results name=train_test2

Ultralytics YOLOv8.0.227 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/content/drive/MyDrive/yolov8/dataset.yaml, epochs=10, patience=50, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/drive/MyDrive/yolov8/training_results, name=train_test2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, 

In [23]:
!yolo task=detect mode=predict model=/content/drive/MyDrive/yolov8/training_results/train_test2/weights/best.pt conf=0.7 source=/content/drive/MyDrive/yolov8/test_images

Ultralytics YOLOv8.0.227 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11129454 parameters, 0 gradients, 28.5 GFLOPs

image 1/1 /content/drive/MyDrive/yolov8/test_images/merlin_193140126_c7444a27-465e-4030-8d74-d04f30be23fc-superJumbo.jpg: 448x640 2 Anesthesia machiness, 66.7ms
Speed: 2.7ms preprocess, 66.7ms inference, 90.6ms postprocess per image at shape (1, 3, 448, 640)
Results saved to runs/detect/predict
💡 Learn more at https://docs.ultralytics.com/modes/predict


In [ ]:
!cp -r /content/runs/detect/predict /content/drive/MyDrive/yolov8/output